In [ ]:
import ROOT as rt
import uproot
import numpy as np
from collections import OrderedDict
import csv
import os
import math
import sys
sys.path.append(os.getcwd().replace('scripts', 'lib'))


import tdrstyle
a = tdrstyle.setTDRStyle()

print(sys.version)

# Load ntuples
open one signal simulation sample and choose interesting events to select from there

In [ ]:
fpath =OrderedDict()
tree = OrderedDict()


mass = [40]
OLD_CTAU = np.array([1000])#in mm
path = '/eos/uscms/store/user/cmsdas/2024/long_exercises/MDS/signal/nocuts/'
for m in mass:
    for ct in OLD_CTAU:
        key = 'MC_'+str(m)+'_'+str(ct)                       
        fpath[key] = path +  'ggH_HToSSTobbbb_MH-125_MS-40_ctau-'+str(ct)+'_137000pb_weighted.root'

for k,v in fpath.items():
    root_dir = uproot.open(v) 
    if not root_dir: 
        print(k, "zombie")
        continue
    tree[k] = root_dir['MuonSystem']
    print(k)

# Selections
Make signal region selections to select for large clusters from LLPs and save the run number, lumi section, and event numbers

In [ ]:
deltaPhi = {}
Nhits = {}
runNum = {}
evtNum = {}
lumiNum = {}


for k, T in tree.items():
    
    ########### SELECTION: CLUSTERS ############

    sel_rechitcluster = np.abs(T['cscRechitClusterEta'].array()) < 2.0

    me1112_veto = 0
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitClusterNRechitChamberPlus11'].array() <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitClusterNRechitChamberPlus12'].array() <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitClusterNRechitChamberMinus11'].array() <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitClusterNRechitChamberMinus12'].array() <= me1112_veto)

    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitCluster_match_MB1Seg_0p4'].array() <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitCluster_match_RE12_0p4'].array() <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitCluster_match_RB1_0p4'].array() <= me1112_veto)

    sel_rechitcluster = np.logical_and( sel_rechitcluster, T['cscRechitClusterJetVetoPt'].array() < 10)
    sel_rechitcluster = np.logical_and( sel_rechitcluster, T['cscRechitClusterMuonVetoPt'].array() < 20)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitClusterTimeSpread'].array() < 20)

    sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_and(T['cscRechitClusterTime'].array() < 12.5, \
                                                                         T['cscRechitClusterTime'].array() > -5.0))

    ########### SELECTION: JETS ############

    sel_jet = np.logical_and(T['jetPt'].array() > 30, np.abs(T['jetEta'].array()) < 2.4 )
    sel_jet = np.logical_and(sel_jet, T['jetTightPassId'].array())

    ########### SELECTION: EVENTS ############
    sel_ev = T['METTrigger'].array()
    sel_ev = np.logical_and(sel_ev ,T['met'].array() > 200)
    sel_ev = np.logical_and(sel_ev,T['metFilters'].array())
    sel_ev = np.logical_and(sel_ev , np.sum(sel_jet,axis=1)>=1)
    sel_ev  = np.logical_and(sel_ev,np.sum(sel_rechitcluster,axis=1) >= 1)
    sel_ev  = np.logical_and(sel_ev,T['MC_condition'].array() == 2017)


    ##### cut based ID ####
    cscRechitClusterNStation = T['cscRechitClusterNStation'].array()[sel_rechitcluster][sel_ev][:,0]
    cscRechitClusterEta = T['cscRechitClusterEta'].array()[sel_rechitcluster][sel_ev][:,0]
    cscRechitClusterAvgStation = T['cscRechitClusterAvgStation'].array()[sel_rechitcluster][sel_ev][:,0]
    cond2 = np.logical_and(np.abs(cscRechitClusterAvgStation)==2, np.abs(cscRechitClusterEta) < 1.6)
    cond3 = np.logical_and(np.abs(cscRechitClusterAvgStation)==3, np.abs(cscRechitClusterEta) < 1.6)
    cond4 = np.logical_and(np.abs(cscRechitClusterAvgStation)==4, np.abs(cscRechitClusterEta) < 1.8)
    cond1 = np.logical_and(cscRechitClusterNStation==1, np.logical_or(np.logical_or(np.abs(cscRechitClusterAvgStation)==1, cond2), np.logical_or(cond3, cond4)))
    cond2 = np.logical_and(cscRechitClusterNStation > 1, np.abs(cscRechitClusterEta) < 1.9)
    bdt_sel = np.logical_or(np.logical_or(cond1, cond2), np.logical_or(cond3, cond4))

    #### Save variables needed ####                                   

    deltaPhi[k] = T['cscRechitClusterMet_dPhi'].array()[sel_rechitcluster][sel_ev][:,0][bdt_sel]
    Nhits[k] = T['cscRechitClusterSize'].array()[sel_rechitcluster][sel_ev][:,0][bdt_sel]
    
    evtNum[k] =  T['evtNum'].array()[sel_ev][bdt_sel]
    runNum[k] = T['runNum'].array()[sel_ev][bdt_sel]
    lumiNum[k] = T['lumiSec'].array()[sel_ev][bdt_sel]
    
    
    
    print(k)

### Now you will write the event/run/lumi number of the events that you want to choose in a text file

In [ ]:
output_text_path = os.getcwd().replace('scripts', 'event_display')
if not os.path.isdir(output_text_path):os.makedirs(output_text_path)
print(output_text_path)

Make the selections that you want to select for events (You can change the `Nhits` threshold to even large numbers to get more interesting events to view!)

The code currently selects for the first 10 events to save, don't change this number to more than 10, the next steps might take significantly longer

In [ ]:
# select events
selection = np.logical_and(Nhits[k]>500, np.abs(deltaPhi[k])<0.75)
print(np.count_nonzero(selection))

text_file = open(output_text_path + "/event_display.txt", "w")

# choose the first 10 events to view, don't change this number to more than 10, 
# the next steps might take significantly longer
for i in range(10):
    print('{}:{}:{}'.format(runNum[k][i], lumiNum[k][i], evtNum[k][i]))
    text_file.write('{}:{}:{} \n'.format(runNum[k][i], lumiNum[k][i], evtNum[k][i]))
text_file.close()